# Basic Learning About YOLO Dataset

### What does YOLO dataset format look like?
```
 data_directory/
    ├── images/
    │   ├── image1.jpg
    │   ├── image2.jpg
    │   ├── ...
    │   └── imageN.jpg
    └── labels/
        ├── image1.txt
        ├── image2.txt
        ├── ...
        └── imageN.txt 


```


### What does label data look like?
```
2 0.561805 0.503788 0.865275 0.983333
0 0.569729 0.083333 0.310611 0.139394
3 0.477813 0.838636 0.329628 0.089394
```

- a row --> an object detecting in this image
- the meaning of each element within each row:
    - label 
    - the bounding box middle point x 
    - the bounding box middle point y
    - the bounding box width
    - the bounding box heigth




# Check GPU Setting
Remember to click Accelerator to activate GPU for the model training, and the you may use `nvydia-smi` to check current status of GPU

In [1]:
! nvidia-smi

Mon Apr 29 08:34:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0              26W / 250W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Install Needed Packages

In [2]:
# install packages used in YOLO model
! pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 752.1/752.1 kB 13.4 MB/s eta 0:00:00a 0:00:01


In [3]:
# check model
import ultralytics
ultralytics.checks() 

Ultralytics YOLOv8.2.4 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5597.6/8062.4 GB disk)


# Data Analytics - EDA
Let's take a look at the overall data pattern for further model training

### Get Data Size, Class Count

In [4]:
from glob import glob
from itertools import chain
from collections import Counter
from pprint import pprint

id2class_map = {
    '0': 'with helmet', 
    '1': 'without helmet', 
    '2': 'rider', 
    '3': 'number_plate'
}

main_path = "/kaggle/input/rider-with-helmet-without-helmet-number-plate"  

def print_data_size(folder_type):
    data_size = len(glob(f'{main_path}/{folder_type}/labels/*.txt'))
    print(f'{folder_type} data count: {data_size}')
    
def print_class_count(folder_type):
    class_list = []
    for file in glob(f'{main_path}/{folder_type}/labels/*.txt'):
        class_list.append([row.split()[0] for row in open(file, "r")])
    counter = Counter(list(chain(*class_list)))
    print(f'-- data class count')
    pprint({f'{k}. {id2class_map[k]}':v for k, v in counter.items()})
    print()

print_data_size('train')
print_class_count('train')
print_data_size('val')
print_class_count('val')  

train data count: 104
-- data class count
{'0. with helmet': 64,
 '1. without helmet': 93,
 '2. rider': 120,
 '3. number_plate': 116}

val data count: 20
-- data class count
{'0. with helmet': 13,
 '1. without helmet': 15,
 '2. rider': 23,
 '3. number_plate': 22}



### Plot Training Images

In [5]:
# plot training images
import os
import cv2
from matplotlib import pyplot as plt

main_path = f"/kaggle/input/rider-with-helmet-without-helmet-number-plate"  

def get_bbox_and_label(image_name, main_path=main_path):
    ''' get bbox and label information from label txt files '''
    
    # read file from path
    lbl_path = os.path.join(main_path, 'train', 'labels', f'{image_name}.txt')
    with open(lbl_path, 'r') as f:
        lines = f.readlines()
    
    # extract bboxes and labels from the label file
    bboxes = [
        [float(n) for n in line.split()[1:]]
        for line in lines
    ]
    labels = [id2class_map[line.split()[0]] for line in lines]
    
    return bboxes, labels
    
def load_image(image_name, main_path=main_path):
    img_path = os.path.join(main_path, 'train', 'images', f'{image_name}.jpg')    
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

def get_bbox_coordinates(img, bbox):
    # reference
    # https://medium.com/@miramnair/yolov7-calculating-the-bounding-box-coordinates-8bab54b97924
    
    img_height, img_width, _ = img.shape
    x_center, y_center, bbox_width, bbox_height = bbox
    
    # calculate the coordinates of the bounding box
    x_center_pixel = x_center * img_width
    y_center_pixel = y_center * img_height
    half_width = bbox_width * img_width / 2
    half_height = bbox_height * img_height / 2
    
    x_min = int(x_center_pixel - half_width)
    y_min = int(y_center_pixel - half_height)
    x_max = int(x_center_pixel + half_width)
    y_max = int(y_center_pixel + half_height)
    
    return x_min, y_min, x_max, y_max

class2color_map = {
    'with helmet': (0,255,128),
    'without helmet': (255,51,51),
    'rider': (51,255,255),
    'number_plate': (255,255,255)
}

def plot_image(image_name, class2color_map=class2color_map):
    img = load_image(image_name)
    bboxes, labels = get_bbox_and_label(image_name=image_name)
    for bbox, label in zip(bboxes, labels):
        
        # get bbox and label info
        color = class2color_map[label]
        x_min, y_min, x_max, y_max = get_bbox_coordinates(img, bbox)
        
        # add bounding box with rectangle
        img = cv2.rectangle(img,(x_min,y_min),(x_max,y_max), color, 2)
        
        # add label info
        img = cv2.putText(
                img,
                label,
                (x_min, y_min + 10),
                fontFace = cv2.FONT_HERSHEY_SIMPLEX,
                fontScale = 0.6,
                color = color,
                thickness=2
            )
    plt.imshow(img)
    plt.show()

In [ ]:
plot_image(image_name='new100')

# Load YOLO8 Model

In [25]:
from ultralytics import YOLO

# Load a COCO-pretrained YOLOv8n model
model = YOLO('yolov9c.pt')

# Display model information (optional)
model.info()

100%|██████████| 49.4M/49.4M [00:00<00:00, 75.6MB/s]


YOLOv9c summary: 618 layers, 25590912 parameters, 0 gradients, 104.0 GFLOPs


(618, 25590912, 0, 104.02268160000003)

# Set yaml & Model Training

In [26]:
import yaml

# read the content of coco128.yaml
with open(os.path.join(main_path, 'coco128.yaml'), 'r') as file:
    print(file.read())

train: /train/images
val: /val/images

nc: 4

names: ["with helmet", "without helmet", "rider", "number plate"]


In [27]:
# creata the yaml based on coco128 for model training
data = {
    'train': '/kaggle/input/rider-with-helmet-without-helmet-number-plate/train',
    'val': '/kaggle/input/rider-with-helmet-without-helmet-number-plate/val',
    'nc': 4,
    'names': [
        'with helmet',
        'without helmet',
        'rider',
        'number plate'
    ]
}

with open('/kaggle/working/data.yaml', 'w') as file:
    yaml.dump(data, file)

# check the content of data.yaml
with open('/kaggle/working/data.yaml', 'r') as file:
    print(file.read())

names:
- with helmet
- without helmet
- rider
- number plate
nc: 4
train: /kaggle/input/rider-with-helmet-without-helmet-number-plate/train
val: /kaggle/input/rider-with-helmet-without-helmet-number-plate/val



In [28]:
# disable wandb
os.environ['WANDB_MODE'] = "disabled"


# command to train the YOLOv8 model
model.train(
    data='/kaggle/working/data.yaml',
    epochs=50,
    workers=1,
    batch=8,
)

Ultralytics YOLOv8.2.4 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
engine/trainer: task=detect, mode=train, model=yolov9c.pt, data=/kaggle/working/data.yaml, epochs=50, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=1, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, s

train: Scanning /kaggle/input/rider-with-helmet-without-helmet-number-plate/train/labels... 102 images, 0 backgrounds, 2 corrupt: 100%|██████████| 104/104 [00:00<00:00, 524.81it/s]

train: WARNING ⚠️ /kaggle/input/rider-with-helmet-without-helmet-number-plate/train/images/new128.jpg: ignoring corrupt image/label: cannot identify image file '/kaggle/input/rider-with-helmet-without-helmet-number-plate/train/images/new128.jpg'
train: WARNING ⚠️ /kaggle/input/rider-with-helmet-without-helmet-number-plate/train/images/new3.jpg: ignoring corrupt image/label: invalid image format GIF. Supported formats are:
images: {'pfm', 'webp', 'tif', 'mpo', 'jpeg', 'jpg', 'png', 'bmp', 'dng', 'tiff'}
videos: {'mkv', 'mpg', 'mpeg', 'mp4', 'avi', 'gif', 'webm', 'mov', 'ts', 'asf', 'm4v', 'wmv'}
train: WARNING ⚠️ Cache directory /kaggle/input/rider-with-helmet-without-helmet-number-plate/train is not writeable, cache not saved.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /kaggle/input/rider-with-helmet-without-helmet-number-plate/val/labels... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<00:00, 514.10it/s]

val: WARNING ⚠️ Cache directory /kaggle/input/rider-with-helmet-without-helmet-number-plate/val is not writeable, cache not saved.


...

                   all         20         73      0.911      0.905      0.926      0.755



50 epochs completed in 0.129 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 51.6MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 51.6MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics YOLOv8.2.4 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
YOLOv9c summary (fused): 384 layers, 25322332 parameters, 0 gradients, 102.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  4.53it/s]


                   all         20         73      0.875        0.9      0.924      0.755
           with helmet         20         13      0.922      0.912      0.952      0.755
        without helmet         20         15      0.747        0.8      0.839      0.641
                 rider         20         23      0.917      0.913      0.915      0.828
          number plate         20         22      0.915      0.977      0.988      0.796
Speed: 0.2ms preprocess, 16.1ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78b5cd86f160>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0

# Check Val Detecting Image
We may check the detection result of our validation images.

In [29]:
# select the best model for checking prediction plot
# the model is saved in best.pt directly after training
model = YOLO('/kaggle/working/runs/detect/train2/weights/best.pt') 

In [30]:
!pip install pytesseract

In [31]:
import pytesseract

In [32]:
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

In [33]:
!pip install ocr

ERROR: Could not find a version that satisfies the requirement ocr (from versions: none)
ERROR: No matching distribution found for ocr


In [34]:
import pytesseract
from PIL import Image

In [35]:
pip install pytesseract Pillow

Note: you may need to restart the kernel to use updated packages.


In [36]:
import csv

In [37]:
def ocr(image):
    # Perform OCR on the image
    number_plate_text = pytesseract.image_to_string(image, config='--psm 6')
    return number_plate_text.strip()

In [38]:
def write_to_csv(data, csv_file):
    with open(csv_file, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(data)

In [39]:
# Define function to plot prediction results and record number plates
def plot_pred_image(image_name, id2class_map=id2class_map, class2color_map=class2color_map):
    image_path = os.path.join(main_path, 'val', 'images', f'{image_name}.jpg')
    
    # Perform inference
    results = model.predict(image_path)
    r = results[0]
    img = r.orig_img
    bboxes = r.boxes.xyxy.tolist()
    labels = [id2class_map[str(int(c))] for c in r.boxes.cls.tolist()]
    
    # Initialize list to store number plates of vehicles with > 2 riders or without helmets
    vehicles_to_record = []
    
    for bbox, label in zip(bboxes, labels):
        # Get bbox and label info
        color = class2color_map[label]
        x_min, y_min, x_max, y_max = [int(n) for n in bbox]
        
        # Add bounding box with rectangle
        img = cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color, 2)
        
        # Add label info
        img = cv2.putText(
            img,
            label,
            (x_min, y_min + 10),
            fontFace=cv2.FONT_HERSHEY_SIMPLEX,
            fontScale=0.6,
            color=color,
            thickness=2
        )
        
        # Check if label is a vehicle ('rider' assumes label for vehicles)
        if label == 'rider':
            # Filter out persons associated with the current vehicle
            persons_on_vehicle = [person_bbox for person_bbox, person_label in zip(bboxes, labels) if person_label == 'rider' and person_bbox[:4] in bbox[:4]]
            
            # Check if the number of riders exceeds 2 or if any rider is without a helmet
            if len(persons_on_vehicle) > 2 or 'without helmet' in labels:
                # Find and record the number plate
                for plate_bbox, plate_label in zip(bboxes, labels):
                        
                        
                    print(plate_label=='number_plate')
                    if plate_label =='number_plate':

                        number_plate_image = img[int(plate_bbox[1]):int(plate_bbox[3]), int(plate_bbox[0]):int(plate_bbox[2])]
                        number_plate_text = ocr(number_plate_image)
                        print(number_plate_text)
                        vehicles_to_record.append({'bbox': plate_bbox, 'text': number_plate_text})
                        break
    
    # Save number plates to CSV if there are any to record
    if vehicles_to_record:
        csv_file = 'recorded_number_plates.csv'
        
        for vehicle in vehicles_to_record:
            write_to_csv([image_name, *vehicle['bbox'], vehicle['text']], csv_file)
    
    # Save the image with predictions
    output_image_path = f'predicted_{image_name}.jpg'
    cv2.imwrite(output_image_path, img)
    print(f"Predictions saved to {output_image_path}")

# Example usage
plot_pred_image(image_name='new73')


image 1/1 /kaggle/input/rider-with-helmet-without-helmet-number-plate/val/images/new73.jpg: 384x640 2 without helmets, 1 rider, 1 number plate, 11.1ms
Speed: 2.7ms preprocess, 11.1ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)
False
False
False
True
— a.e
qe0g: 7 :9699
Predictions saved to predicted_new73.jpg


In [40]:
import pandas as pd


In [41]:
df = pd.read_csv('/kaggle/working/runs/detect/train2/results.csv')
# df = pd.read_csv('/kaggle/working/recorded_number_plates.csv')
df

,epoch,train/box_loss,train/cls_loss,train/dfl_loss,metrics/precision(B),metrics/recall(B),metrics/mAP50(B),metrics/mAP50-95(B),val/box_loss,val/cls_loss,val/dfl_loss,lr/pg0,lr/pg1,lr/pg2
0,1,1.09620,3.47910,1.29440,0.01890,0.39682,0.21793,0.15926,0.74272,3.60930,1.03620,0.000150,0.000150,0.000150
1,2,0.90777,2.50420,1.14310,0.02192,0.40452,0.21906,0.17398,0.79953,3.26960,1.00310,0.000306,0.000306,0.000306
2,3,0.89555,1.52750,1.13780,1.00000,0.11853,0.72198,0.56885,0.84729,2.71430,1.05670,0.000456,0.000456,0.000456
3,4,0.89079,1.31850,1.08820,0.91045,0.39604,0.83292,0.62703,1.02790,2.14160,1.17380,0.000600,0.000600,0.000600
4,5,0.91977,1.25120,1.09820,0.82590,0.57423,0.85205,0.62986,0.96617,2.05970,1.06570,0.000737,0.000737,0.000737
5,6,0.86709,1.17230,1.05580,0.74936,0.76126,0.89301,0.65382,0.91468,1.93570,1.04750,0.000867,0.000867,0.000867
6,7,0.86743,1.13490,1.08770,0.81379,0.63803,0.87816,0.64393,0.92887,1.95570,1.10530,0.000991,0.000991,0.000991
7,8,0.85997,1.10320,1.06960,0.83140,0.53640,0.86342,0.64792,0.94441,2.05400,1.12020,0.001077,0.001077,0.001077
8,9,0.86996,1.03800,1.07380,0.90020,0.66594,0.88241,0.62328,0.93061,1.79910,1.10970,0.001052,0.001052,0.001052
9,10,0.84100,1.01180,1.05140,0.82040,0.66926,0.86202,0.62586,0.92862,1.71220,1.14740,0.001027,0.001027,0.001027
